
# Linear Relaxation of Softsign

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Activation("softsign", input_dim=1))

In [ ]:
import sys

sys.path.append("../..")
from decomon.models import convert

In [ ]:
decomon_model = convert(model)

In [ ]:
def softsign(x):
    return x / (np.abs(x) + 1.0)

In [ ]:
nn_layers = [Dense(10, activation="relu", input_dim=1), Dense(10, activation="relu"), Dense(1)]

nn_model = Sequential(nn_layers)

In [ ]:
nn_model.compile("adam", "mse")

In [ ]:
x_ = np.linspace(-10, 10, 1000)
nn_model.fit(x_, model.predict(x_), epochs=10)

In [ ]:
def linear_hull_upper_decomon(lower, upper):
    x_min = lower * np.ones((1, 1))
    x_max = upper * np.ones((1, 1))
    box = np.concatenate([x_min[:, None], x_max[:, None]], 1)
    # print(box)
    output = decomon_model.predict([x_max, box])
    _, _, upper, _, w_u, b_u, _, _ = output

    return upper.min(), w_u.min(), b_u.min()


def linear_hull_lower_decomon(lower, upper):
    x_min = lower * np.ones((1, 1))
    x_max = upper * np.ones((1, 1))
    box = np.concatenate([x_min[:, None], x_max[:, None]], 1)

    output = decomon_model.predict([x_min, box])
    _, _, _, lower, _, _, w_l, b_l = output
    return lower.min(), w_l.min(), b_l.min()

In [ ]:
def func(x_min, x_max):
    n_step = 100
    if x_max < x_min:
        x_max_ = x_max + 0
        x_max = x_min
        x_min = x_max_
    x = np.linspace(x_min, x_max, n_step)
    y = np.linspace(-10, 10, 1000)

    _, w_u, b_u = linear_hull_upper_decomon(x_min, x_max)
    _, w_l, b_l = linear_hull_lower_decomon(x_min, x_max)

    s_z = nn_model.predict(x)
    s_x = softsign(x)
    s_y = softsign(y)
    z_x = w_u * x + b_u
    q_x = w_l * x + b_l

    plt.ylim(-1.3, 1.3)

    plt.plot(x, s_x, c="blue")
    plt.plot(x, z_x, c="orange")
    plt.plot(x, q_x, c="green")
    plt.plot(x, s_z, c="pink")

    plt.plot(y, s_y, "--", c="k")
    plt.title("Linear Relaxation of Softsign")

In [ ]:
interact(
    func,
    x_min=widgets.FloatSlider(value=-10, min=-10, max=10.0, step=0.01, continuous_update=False),
    x_max=widgets.FloatSlider(value=10, min=-10, max=10, step=0.01, continuous_update=False),
)